In [ ]:
import logging
import gym
import numpy as np
import os
import matplotlib.pyplot as plt

# add reference libraries here. Current structure will use the relative path from this file
libs = ["NonlinearPendulumTest.mo"]

# check that all the paths to library package.mo files exist
# DymolaInterface() also checks this but I've found this warning helpful
for lib in libs:
    if not os.path.isfile(lib):
        print(f"Cannot find the library {lib}")

mo_name = "NonlinearPendulumTest" # name of Modelica model in the Library.Model format
env_entry_point = 'examples:PendulumEnvSEM' # Python package location of RL environment

time_step = 0.5 # time delta in seconds
positive_reward = 1
negative_reward = -100 # penalize RL agent for is_done
log_level = 0
default_action = []

# these config values are passed to the model specific environment class
# mo_name and libs are passed on to the DymolaBaseEnv class
config = {
    'mo_name': mo_name,
    'libs': libs,
    'time_step': time_step,
    'positive_reward': positive_reward,
    'negative_reward': negative_reward,
    'log_level': log_level,
    'default_action': default_action,
    'method':'Dassl'
}

# enable the model specific class as an OpenAI gym environment
from gym.envs.registration import register
env_name = "PedulumEnv-v0"

register(
    id=env_name,
    entry_point=env_entry_point,
    kwargs=config
)

env = gym.make(env_name)

In [ ]:
args = ['random','not_random']
colors = ['orange', 'blue']
legend = []
for arg in args:
    # reset the model 
    env.reset_dymola()
    env.reset()
    env.debug_data = {key: [] for key in env.debug_data}
    
    # see what the model outputs
    for timestep in range(5):
        if arg == 'random':
            action = np.random.uniform(1,2)
        else:
            action = 1.5
        env.step([action])
        
    c = colors.pop()
    plt.plot(env.debug_data['nonlinearPendulum.my_time'], env.debug_data['nonlinearPendulum.theta'], linestyle='-', color=c)
    plt.plot(env.debug_data['nonlinearPendulum.my_time'], env.debug_data['nonlinearPendulum.omega'], linestyle='--', color=c)
    legend += [f"{arg}, theta", f"{arg}, omega"]
plt.legend(legend)

In [ ]:
args = ['random','not_random']
colors = ['orange', 'blue']
legend = []
for arg in args:
    # reset the model 
    env.reset_dymola()
    env.reset()
    env.debug_data = {key: [] for key in env.debug_data}
    
    # see what the model outputs
    for timestep in range(5):
        if arg == 'random':
            action = np.random.uniform(1,2)
        else:
            action = 1.5
        
        env.dymola.importInitialResult('dsres.mat', atTime=env.start)
        res = env.dymola.simulateExtendedModel(env.model_name, startTime=env.start, stopTime=env.stop, initialNames=env.model_input_names, initialValues=[action], finalNames=env.model_output_names)
        env.get_state(res) # adds intermittent values to the debug_data dictionary
        env.start += env.tau
        env.stop += env.tau
    c = colors.pop()
    plt.plot(env.debug_data['nonlinearPendulum.my_time'], env.debug_data['nonlinearPendulum.theta'], linestyle='-', color=c)
    plt.plot(env.debug_data['nonlinearPendulum.my_time'], env.debug_data['nonlinearPendulum.omega'], linestyle='--', color=c)
    legend += [f"{arg}, theta", f"{arg}, omega"]
plt.legend(legend)

In [ ]:
env.dymola.close()